In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)  # randomness of response

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(
    country_a="Mexico",
    country_b="Korea",
)

chat.predict(prompt)

'The distance between Mexico and Korea is approximately 11,000 kilometers (6,835 miles).'

In [2]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a geography expert. And you only reply in {language}.",
        ),
        (
            "ai",
            "Ciao, mi chiamo {name}!",
        ),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="Greek", name="Socrates", country_a="Mexico", country_b="Korea"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σου! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Κορέας εξαρτάται από το ποια πόλη του Μεξικού και ποια πόλη της Κορέας αναφέρεσαι. Συνήθως, η απόσταση ανάμεσά τους είναι περίπου 13.000 χιλιόμετρα.')

In [3]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

In [17]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list geterating machine.Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else.",
        ),
        (
            "human",
            "{question}",
        ),
    ]
)

['mercury',
 'venus',
 'earth',
 'mars',
 'jupiter',
 'saturn',
 'uranus',
 'neptune',
 'pluto']

In [18]:
chain = template | chat | CommaOutputParser()

chain.invoke(
    {
        "max_items": 5,
        "question": "What are the pokemons?",
    }
)

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']